In [1]:
# !pip install mlflow

In [2]:
# !pip install xgboost

In [1]:
import pandas as pd
import mlflow
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score, \
                            roc_curve,confusion_matrix

In [2]:
data = pd.read_csv('process.csv', index_col=0)
data.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,1,2,1,6,142,4,3,3,3,2,...,3,48,1,1,1,1,0,1,1,1
1,0,29,3,6,770,2,0,1,0,2,...,3,3,1,1,0,1,0,0,1,0
2,3,8,1,2,390,2,1,1,3,2,...,3,30,1,1,0,3,1,0,1,1
3,1,26,3,3,848,2,1,1,3,1,...,1,26,1,0,0,1,1,0,1,1
4,1,17,2,4,734,2,0,2,3,2,...,2,34,1,0,1,1,1,0,1,0


In [3]:
# Independent and dependent data
x = data[data.columns.difference(['class'])]
y = data['class']

In [7]:
# Scaler data
scaler =  StandardScaler()
Xscale  = scaler.fit_transform(x)
Xscale

array([[ 2.78850999, -0.4645937 , -1.15669474, ...,  0.62648137,
         1.04698668,  1.66990103],
       [-1.19773321, -1.26286467,  1.21764107, ...,  0.62648137,
        -0.76597727, -0.13053135],
       [ 1.19401271,  1.13194823, -0.21905894, ..., -1.02650377,
         0.14050471, -0.13053135],
       ...,
       [ 0.21959771,  1.13194823, -1.43269237, ...,  0.62648137,
         1.04698668, -0.13053135],
       [-1.10915003, -0.4645937 , -0.43834473, ...,  0.62648137,
         1.04698668, -0.13053135],
       [-0.7548173 , -1.26286467,  0.99457449, ...,  1.86622023,
         1.04698668, -1.93096373]])

In [9]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(Xscale, y, test_size = 0.20, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 20), (200, 20), (800,), (200,))

In [41]:
# local notebook mlflow 
# mlflow.set_tracking_uri("sqlite:///mlflow.db")
# mlflow.set_experiment("xgboost-ml")

# mlflow.client
# from mlflow.tracking import MlflowClient
# client = MlflowClient("sqlite:///mlflow.db")

In [43]:
# mlflow ui setup
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [45]:
# first run
mlflow.set_experiment("xgboost-ml")
xgboost_params = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
kf = KFold(n_splits=30)
           
with mlflow.start_run():
    mlflow.set_tag("model","XGBClassifier")
    mlflow.log_param("xgboost_params",xgboost_params)

    randomcv_models = XGBClassifier()

    random = RandomizedSearchCV(estimator=randomcv_models,
                                        param_distributions=xgboost_params,
                                       n_iter=100,
                                       cv=kf,
                                       verbose=2, 
                                       n_jobs=-1)
    random.fit(X_train, y_train)
    print(random.best_params_)
    
    y_pred = random.predict(X_test)

    mlflow.sklearn.log_model(random, artifact_path="models")

2024/07/22 14:52:37 INFO mlflow.tracking.fluent: Experiment with name 'xgboost-ml' does not exist. Creating a new experiment.
C:\Users\sudwa\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 30 folds for each of 12 candidates, totalling 360 fits
{'min_child_weight': 5, 'max_depth': 7}


In [47]:
# first search
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1721685157391, experiment_id='3', last_update_time=1721685157391, lifecycle_stage='active', name='xgboost-ml', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1721174520861, experiment_id='2', last_update_time=1721174520861, lifecycle_stage='active', name='my-experiment-3', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1721174454461, experiment_id='1', last_update_time=1721174454461, lifecycle_stage='active', name='my-experiment-2', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721174343796, experiment_id='0', last_update_time=1721174343796, lifecycle_stage='active', name='Default', tags={}>]

In [50]:
#second run
mlflow.set_experiment("xgboost-ml-2")

xgboost_params = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
kf = KFold(n_splits=30)
           
with mlflow.start_run():
    mlflow.set_tag("model","XGBClassifier")
    mlflow.log_param("xgboost_params",xgboost_params)

    randomcv_models = XGBClassifier()

    random = RandomizedSearchCV(estimator=randomcv_models,
                                        param_distributions=xgboost_params,
                                       n_iter=100,
                                       cv=kf,
                                       verbose=2, 
                                       n_jobs=-1)
    random.fit(X_train, y_train)
    print(random.best_params_)
    
    y_pred = random.predict(X_test)

    mlflow.sklearn.log_model(random, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/07/22 14:56:20 INFO mlflow.tracking.fluent: Experiment with name 'xgboost-ml-2' does not exist. Creating a new experiment.
C:\Users\sudwa\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 30 folds for each of 12 candidates, totalling 360 fits
{'min_child_weight': 5, 'max_depth': 7}
default artifacts URI: 'mlflow-artifacts:/4/2500e761510d443088845f7cb633db35/artifacts'


In [52]:
# Second search
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1721685380881, experiment_id='4', last_update_time=1721685380881, lifecycle_stage='active', name='xgboost-ml-2', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1721685157391, experiment_id='3', last_update_time=1721685157391, lifecycle_stage='active', name='xgboost-ml', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721174343796, experiment_id='0', last_update_time=1721174343796, lifecycle_stage='active', name='Default', tags={}>]

In [54]:
#third run
mlflow.set_experiment("xgboost-ml-3")
xgboost_params = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
kf = KFold(n_splits=30)

with mlflow.start_run():
    mlflow.set_tag("model","XGBClassifier")
    mlflow.log_param("xgboost_params",xgboost_params)

    randomcv_models = XGBClassifier()

    random = RandomizedSearchCV(estimator=randomcv_models,
                                        param_distributions=xgboost_params,
                                       n_iter=100,
                                       cv=kf,
                                       verbose=2, 
                                       n_jobs=-1)
    random.fit(X_train, y_train)
    print(random.best_params_)
    
    y_pred = random.predict(X_test)

    mlflow.sklearn.log_model(random, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/07/22 14:59:12 INFO mlflow.tracking.fluent: Experiment with name 'xgboost-ml-3' does not exist. Creating a new experiment.
C:\Users\sudwa\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 30 folds for each of 12 candidates, totalling 360 fits
{'min_child_weight': 5, 'max_depth': 7}
default artifacts URI: 'mlflow-artifacts:/5/55ec22ab2e02409aac217278a8db3d3a/artifacts'


In [57]:
# Third search
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1721685552484, experiment_id='5', last_update_time=1721685552484, lifecycle_stage='active', name='xgboost-ml-3', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1721685380881, experiment_id='4', last_update_time=1721685380881, lifecycle_stage='active', name='xgboost-ml-2', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1721685157391, experiment_id='3', last_update_time=1721685157391, lifecycle_stage='active', name='xgboost-ml', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1721174343796, experiment_id='0', last_update_time=1721174343796, lifecycle_stage='active', name='Default', tags={}>]

In [113]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "xgboost-ml"

# latest_versions = client.get_latest_versions(model_name)

latest_versions = client.get_experiment_by_name(name=model_name)
for version in latest_versions:
    print(version)
    

('artifact_location', 'mlflow-artifacts:/3')
('creation_time', 1721685157391)
('experiment_id', '3')
('last_update_time', 1721685157391)
('lifecycle_stage', 'active')
('name', 'xgboost-ml')
('tags', {})


In [125]:
# model_name = "xgboost-ml"
# model_version = 3
# new_stage = "Staging"
# client.transition_model_version_stage(
#     name=model_name,
#     version=model_version,
#     stage=new_stage,
#     archive_existing_versions=False
# )

In [127]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [59]:
# interacting model registry

In [67]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

run_id=client.get_experiment(experiment_id='3')

print(run_id)

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1721685157391, experiment_id='3', last_update_time=1721685157391, lifecycle_stage='active', name='xgboost-ml', tags={}>


In [69]:
id = client.create_registered_model(name='xgboost-ml-3')
print(id)

<RegisteredModel: aliases={}, creation_timestamp=1721686254663, description='', last_updated_timestamp=1721686254663, latest_versions=[], name='xgboost-ml-3', tags={}>


In [ ]:
# model-version
client = MlflowClient()
result = client.create_model_version(
    name='xgboost-ml-3')

In [ ]:
# final aws ec2 main script file

In [30]:
# Hypertunning Random_search 
xgboost_params = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}

kf = KFold(n_splits=30)

model_param = {}

randomcv_models = XGBClassifier()

random = RandomizedSearchCV(estimator=randomcv_models,
                                    param_distributions=xgboost_params,
                                   n_iter=100,
                                   cv=kf,
                                   verbose=2, 
                                   n_jobs=-1)
random.fit(Xscale, y)
model_param = random.best_params_
print(model_param)


Fitting 30 folds for each of 12 candidates, totalling 360 fits
{'min_child_weight': 3, 'max_depth': 5}


In [23]:
params = {'min_child_weight': 3, 'max_depth': 5}

best_model = XGBClassifier(**params)
best_model = best_model.fit(X_train,y_train)
y_pred = best_model.predict(X_test)
score = accuracy_score(y_test,y_pred)
cr = classification_report(y_test,y_pred)

print("FINAL MODEL 'XGBoost'")
print ("Accuracy Score value: {:.4f}".format(score))
print (cr)

FINAL MODEL 'XGBoost'
Accuracy Score value: 0.8200
              precision    recall  f1-score   support

           0       0.73      0.61      0.67        59
           1       0.85      0.91      0.88       141

    accuracy                           0.82       200
   macro avg       0.79      0.76      0.77       200
weighted avg       0.81      0.82      0.81       200

